# Tinkering Notebook - Function Approximation 

# Table of content
* ### [1. Introduction <a id="intro">](#sec1)
* ### [2. Imports <a id="imports">](#sec2)
* ### [3. Value-Function Approximation](#sec3)
 * #### [3.1 Linear Approximation with Polynomials and Fourier Basis ](#sec3_1)
 * #### [3.2 Monte-Carlo with value function approximation](#sec3_2)
 * #### [3.3 Run the agent](#sec3_3)
 * #### [3.4 Discussions](#sec3_4)
* ### [4. On-Policy Control with Approximation](#sec4)
 * #### [4.1 Linear Approximation with Tile Coding ](#sec4_1)
 * #### [4.2 SARSA for Estimating Action-Value Function with Function Approximation](#sec4_2)
 * #### [4.3 Run the agent](#sec4_3)
 * #### [4.4 Discussions](#sec4_4)


# 1. Introduction <a id="intro"> <a id="sec1">

References to examples, figures and pages are to the Reinforcement Learning book by Sutton and Barto.

This notebook focuses on function approximation for reinforcement learning. We will first illustrate usage of function approximation for prediction of value function. We will use the random walk example from Chapter 9 of the book and use polynomials (Section 9.5.1) and Fourier basis (Section 9.5.2) as features.  We will then illustrate usage of function approximation for prediction of action-value function and use it for control with Mountain-Car environment as in Chapter 10 of the book.  

__Corridor Environment:__ In order to run the exampmles with the corridor environment (1D Random Walk environment of Example 9.1-Example 9.2), you need gym-RLcourse. If you have not installed gym-RLcourse before,  you can install it as follows: 

```bash
git clone https://github.com/ozayca/gym-RLcourse.git
cd gym-RLcourse
pip3 install -e .
```
     
__RandomWalk_100.pickle:__ The file named *RandomWalk_100.pickle* is needed for plotting the true value function for the corridor environment. It should be placed in the same folder with this notebook.     

__Tile Coding:__ __IMPORTANT: DO NOT SKIP!__ For the example with Mountain-Car, we will use tile coding as features, see Section 9.5.4 of the book and  http://incompleteideas.net/tiles/tiles3.html for details. The code for tile coding is provided here: http://incompleteideas.net/tiles/tiles3.py-remove. Since this is not our code, we will not be distributing it. __You need to copy and paste all of the python code on this page into a file named *TileCoding.py*__ and put it into the same folder with this Jupyter Notebook in order to run the control example. 

__Note on Code Completition Tasks__: These tasks are tagged with TODO-STD within the code. The explanation for these taks are given in the text just before the associated code. There are also related small tasks,  see for instance VA1-VA3 just before VA4. These small tasks  are designed to help you to understand crucial concepts and variables/methods.  In most cases, it is not possible to perform the code completion task without understanding these concepts/variables. Please do not try to skip these small tasks! 


# 2. Imports <a id="imports"> <a id="sec2">

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import gym_RLcourse
import pickle # used to read pickle data
import TileCoding as tc
from mpl_toolkits.mplot3d import Axes3D # used for 3D cost-to-go plots

Let's create a corridor: 

In [ ]:
env = gym.make('Corridor-v0', n_starting_states=10, max_delta=2)
print('State space:', env.observation_space)
print('Action space:', env.action_space)

In the corridor environment,  there are `n_starting_states` non-terminal states. The available actions are up to `max_delta` steps to the right and up to `max_delta` steps to the left. There are two terminating states, one in the left and one in the right. Transition into the left and right terminating states gives a reward of -1 of 1, respectively. All other steps give a reward of 0. If an agent chooses actions uniformly randomly from the `2 max_delta` possible actions (i.e. the probability of selection any action is the same), the agent performs a random walk on the corridor. With `n_starting_states =1000`, and `max_delta=100`, this setting corresponds to the 1000-state Random Walk setting of Example 9.1-9.2. 

Let's make a basic check on our installation for the corridor environment. When you run the code, you will see `n_starting_states` non-terminal states,  a terminal state on the left (TL) and another terminal state on the right (TR). Agent's location is indicated by `x` which is (approximately) halfway in the corridor at initialization. Other states are marked with o.


In [ ]:
env.render()

# 3. Value-Function Approximation <a id="sec3">

 We now consider the prediction problem using the parametric approximation of the value function $\hat{v}(s,w)$ with the goal of obtaining  $\hat{v}(s,w)\approx v_\pi(s)$. 

## 3.1 Linear Approximation with Polynomials and Fourier Basis  <a id="sec3_1">

We first implement a linear approximator that uses either  polynomials or Fourier basis as features. 
      
__Task-VA1:__ Below, variable `method` determines whether we use Fourier features or polynomial features. Compare `construct_basis` with the equations in Section 9.5.1 and Section 9.5.2 to understand how the features are implemented. The code uses `lambda` expressions, which is just another way to define functions. You can read about `lambda` expressions in Section 4.7.6 from here https://docs.python.org/3/tutorial/controlflow.html. With this implementation, we obtain a dictionary of basis functions which we can then evaluate for any value of state. 

__Task-VA2:__ Find which methods/variables in the code give the following: i)  $x(s)$ for a given state ii) $\hat{v}(s,w)$ for a given state. 
    
__Task-VA3:__ Consider the approximate value function $\hat{v}(s,w) = w^T x(s)$, where $w$ are the weights and $x(s)$ is the feature vector corresponding to state $s$. Show that the gradient of $\hat{v}(s,w)$ with respect to $w$ is given by $x(s)$, i.e. $\nabla \hat{v}(s,w) = x(s)$.
    
__Task-VA4:__ Complete the `update` function so that it implements Eqn. 9.7.  
   
__Task-VA5:__ In the implementation below, we have a variable called `StateRange` which normalizes the values of the states. Why do you think we may found it useful to include such a variable?



In [ ]:
class LinearApproximator:

    def __init__(self, order, method="Fourier", alpha=1e-4, stateRange=1):
        self.alpha = alpha
        self.method = method
        self.order = order
        self.weights = np.zeros(order + 1)
        self.stateRange = stateRange
        self.construct_basis()
 
    def construct_basis(self):
        self.basis=[]
        if self.method == "Polynomial":
            for i in range(0, self.order + 1):
                self.basis += [lambda s, i=i: np.power(s, i)]
        if self.method == "Fourier":
            for i in range(0, self.order + 1):
                self.basis += [lambda s, i=i: np.cos(np.pi * s * i) ]

    # return the values of features for a given state 
    def features(self, state): 
        state = state / self.stateRange  # normalize to [0, 1]
        return np.array([f(state) for f in self.basis])

    # return the value of the approximation for the given state
    def value(self, state):  
        return np.dot(self.features(state), self.weights)

    # update the weights
    def update(self, target, state):  # update the weights using the eqn 9.7 where U_t is the target
        # TODO-STD

## 3.2 Monte-Carlo with value function approximation <a id="sec3_2">



We now implement the Gradient Monte Carlo Algorithm on Page 202 using a random policy (i.e. all possible actions have the same probability).  

**Task-MC**: Implement the missing parts of the `learn` method which are marked with TODO-STD. 

In [ ]:
class MCAgent:
    # Accepts 'plain' or any method that LinearApproximator accepts. If the method is 'plain',
    # no function approximation is performed. The 'plain' option is included for comparison purposes.
    
    def __init__(self, n_states, n_actions, gamma, method, order, alpha, stateRange):
        self.n_actions = n_actions
        self.V = np.zeros(n_states)
        self.S = np.zeros(n_states)
        self.N = np.zeros(n_states)
        self.n_states = n_states
        self.gamma = gamma
        self.method = method 
        self.func_approx = LinearApproximator(order, method, alpha, stateRange)

    def act(self, state):
        return np.random.choice(self.n_actions)

    def learn(self, states, actions, rewards):
        T = len(states) - 1
        if self.method == "plain":   #  function approximation is NOT performed
            G = 0
            for t in reversed(range(T)):
                G = self.gamma * G + rewards[t + 1]  # G_t
                self.N[states[t]] += 1
                self.V[states[t]] += 1 / self.N[states[t]] * (G - self.V[states[t]])
        else:  #  function approximation is performed
            # TODO-STD Using the arrays of states, actions, rewards, write the code for the following: 
            #  1) implement the last two lines of MC algorithm on page 202.  
            #  2) update self.V[state] so that it gives the approximate value for all states
            # HINTS: 
            # -- Our solution uses methods of LinearApproximator, which are called using self.func_approx.update(), self.func_approx.value()
            # using appropriate inputs. 
            # -- Note about indexing:  
            # The train function will follow the indexing in the book. 
            # Hence, S_t, A_t, R_t+1 are at the indices t, t, t+1 in their respective arrays. 
            
            
            
                
                
            
                
            
                

## 3.3 Run the agent <a id="sec3_3">



We now create the environment. The code run time needed for obtaining illustrative results depends on the environment parameters.  We use the below parameters since they allow us to illustrate all the fundamental aspects we would like to explore in a reasonable time. After experimenting with these values, you can run the code with  `n_starting_states =1000`, and `max_delta=100` to obtain exactly the same environment setting in Example 9.1-9.2 and Figure 9.5. 

In [ ]:
env = gym.make('Corridor-v0', n_starting_states=100, max_delta=10)
stateRange = env.observation_space.n - 2
agent = MCAgent(env.observation_space.n, env.action_space.n, gamma=1, method="Fourier", order=5, alpha=5e-5,
                stateRange=stateRange)

We now run the agent on the environment. __Note that this may take some time, wait until you see 'Finished'.__ 

In [ ]:
nEpisode = 5000
max_nStep = int(2.0e6)
print('Run starts')
for iEpisode in range(1,nEpisode+1):
    if iEpisode % 500 == 0:
        print('Episode', iEpisode)
    state = env.reset()
    stateA = [state]
    action = agent.act(state)
    actionA = [action]
    rewardA = [0]
    done = False
    t_done = -1
    while not done:
        state, reward, done, info = env.step(action)
        stateA.append(state)
        rewardA.append(reward)
        action = agent.act(state)
        actionA.append(action)
        if t_done > max_nStep:
            done = True
        t_done += 1
    agent.learn(stateA, actionA, rewardA)
print('Finished')

We now compare the learned value function with the true value function. 

__Note:__ The correct value function is provided for the environment parameters used in this notebook. Hence, it is not the same with the one provided Figure 9.1 which is for __1000__ non-terminal states. 

__Task-TVF:__ Suggest a method to find the true value function using our knowledge of the environment. 

In [ ]:
with open('RandomWalk_100.pickle', 'rb') as file: # load the pre-calculated true values
    v_true = pickle.load(file)

plt.plot(v_true[1:-1],label='True')
plt.plot(agent.V[1:-1],label='Prediction')
plt.grid(True)
plt.xlabel('States')
plt.ylabel('Value Function')
plt.ylim(-1.1, 1.1)
plt.legend()
plt.show()

## 3.4 Discussions <a id="sec3_4">



We will now compare our results with the ones in Figure 9.5. Nevertheless, note that we  should not expect to obtain exactly the same results with the book. In particular, note the following: i) We do not have any averaging over multiple runs, hence there is considerable randomness in our results. (Hence, after your first experiments, changing the code to average out multiple runs can be a good idea )  ii) Our corridor is a scaled-down version of the one in the book. 


__Task-PD1__:   An important aspect is the definition of $\overline{VE}$ in the book, see Eqn 9.1.  A typical realization of the state distribution under a random agent can be seen in Figure 9.1. According to the Figure 9.1, value function estimates of which states affect $\overline{VE}$ more? State whether you think this is a good way of defining the error.  

__Task-PD2__:   Relate $\overline{VE}$ with  $E_\pi[(v_\pi(S)−\hat{v}(S,w))^2]$ defined on the lecture slides. 

__Task-PD3__:   Repeat the experiment with Fourier basis and Polynomial basis with the order of 5, 10 and 20  and multiple times.  In terms of the general trends, are the results consistent with Figure 9.5? In particular, check the following: 
<ul>
<li> The book states that polynomials do not provide satisfactory results in RL tasks. The results in Figure 9.5 support this claim: Fourier basis shows better performance than polynomials. Are our results consistent with these claims? </li>
<li> Do  higher orders provide significantly better predictions?</li>
</ul>

__Task-PD4__: Run the usual MC agent without function approximation using the option `method= 'plain'`. How does the value function prediction found in this case compare with the predictions using approximation? Which method(s) are *better*? Do your conclusions depend on the length of the corridor and the maximum allowed movement? 



# 4. On-Policy Control with Approximation <a id="sec4">

We focus on the control problem using the parametric approximation of the action-value function $\hat{q}(s,a,w)$ with the aim of obtaining $\hat{q}(s,a,w) \approx q_*(s,a)$. 

Consider Eqn 9.7. We now would like to now approximate the action-value function $q(s,a)$ instead of the value function $v(s)$. Hence, the update will be the following: 

$$ w_{t+1} = w_t + \alpha [U_t - \hat{q}(S_t, A_t, w_t)] \nabla \hat{q}(S_t, A_t, w_t)$$
    
We will refer to this eqn as Eqn 9.7Q. 

## 4.1 Linear Approximation with Tile Coding  <a id="sec4_1">



We now implement a linear approximator that uses tile coding. Since tile coding operates in a slightly different manner than Fourier basis/polynomial, we provide a new linear approximator class for ease of exposition. Note that here indices of active tiles are used to represent value function of different states. See Section 9.5.4 of the book and  http://incompleteideas.net/tiles/tiles3.html for details. 


    
__Task-TC1:__ Examine the usage of tile coding and make sure that you understand how we use the indices of active tiles as features. In particular, check the method `indicesActiveTiles`.

__Task-TC2:__ Consider the method `value`. Verify that this method gives the approximation $\hat{q}(S_t, A_t, w_t) $ for a given state, action pair. 

__Task-TC3:__ Tile coding is just another set of features. Explain why the `value` method looks different than the one for Fourier basis and polynomials.  
    
__Task-TC4:__ Complete the `update` function using Eqn 9.7Q (This eqn is defined in this notebook). Note that this `update` function may look different than the `update' function we had for Fourier basis and polynomials. 





In [ ]:
class LinearApproximatorTile:
    def __init__(self, nTiling=8, size=4096, alpha=0.3, stateRange=1):
        self.size = size
        self.nTiling = nTiling
        self.iht = tc.IHT(size)
        self.weights = np.zeros(size) 
        self.alpha = alpha
        self.stateRange = stateRange

        # scaling for the states, see ``Fleshing out the example" section on http://incompleteideas.net/tiles/tiles3.html
        # For the mountain-car example also see footnote 1 on page 246 of SuttonBarto_2018
        scaleFactor = self.nTiling
        self.scale = [scaleFactor / s for s in self.stateRange]

    # get indices of active tiles (i.e features)
    def indicesActiveTiles(self, state, action):
        scaledState = [s * scale for s, scale in zip(state, self.scale)]
        return tc.tiles(self.iht, self.nTiling, scaledState, [action])

    # calculate q_app(state, action)
    def value(self, state, action):
        ind = self.indicesActiveTiles(state, action)
        return np.sum(self.weights[ind])

    # update the weights 
    def update(self, target, state, action):  # update the weights using  Eqn 9.7Q
        # TODO-STD

## 4.2 SARSA for Estimating Action-Value Function with Function Approximation <a id="sec4_2">

 The following class implements the algorithm *Episodic semi-gradient Sarsa for estimating $q_*$*  on page 244 of the book. 

 Task-Sarsa: Complete the `learn` method. 

In [ ]:
class SARSA:
    def __init__(self, gamma, epsilon, alpha, stateRange):
        self.actions = range(3)  # 0,1,2:  reverse, stay, go forward
        self.state = [0, 0]  # position, velocity
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        self.func_approx = LinearApproximatorTile(nTiling=8, size=4096, alpha=alpha, stateRange=stateRange)
        self.done = False

    def act(self, state): 
        if np.random.random_sample() <= self.epsilon:  # random action wp epsilon  
            action = np.random.choice(self.actions)
        else:  # greedy action wp 1-epsilon
            action_values = []
            for action in self.actions:
                action_values.append(self.func_approx.value(state, action))
            max_value = np.max(action_values)
            indices_bestValue = [i for i, j in enumerate(action_values) if j == max_value]
            action = np.random.choice(indices_bestValue)
        return action

    # Sutton, Barton pg.244
    def learn(self, state, action, reward, state_next, action_next, doneEpisode): 
        # TODO-STD Implement the weight updates according to Sarsa pseudo-code on page 244
        # Note that you need to think of the cases the episode is finished or not seperately
        # HINT: Our implementation uses self.func_approx.value() and self.func_approx.update() with appropriate inputs
        if doneEpisode == True:   #  state_next is a terminal state
           # ....
        else: 
             # ....
   


    # Calculate cost-to-go 
    # With a reward of -1 each step, this corresponds to expected minimum number of steps 
    # required for reaching the goal. See, for instance, Fig 10.1, pg 245.
    def costToGo(self, state):
        q = []
        for action in self.actions:
            q.append(self.func_approx.value(state, action))
        return -np.max(q)

    # !Caution: This plot function assumes a 2-D state space
    def plot_costToGo(self, xA, yA, x_label, y_label):
        xsA, ysA, zsA = [], [], []  # initialize with empty lists
        for x in xA:
            for y in yA:
                state = [x, y]
                xsA.append(x)
                ysA.append(y)
                zsA.append(self.costToGo(state))

        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(xsA, ysA, zsA)

        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.set_zlabel("Cost-to-go")
        plt.show()


We now introduce the training function. 

In [ ]:
def train(env, agent, nEpisode):
    total_reward_episodes = np.zeros(nEpisode)
    max_nStep = 1000
    print('Run starts')
    for iEpisode in range(nEpisode):
        if iEpisode % 50 == 0:
            print('Episode', iEpisode)
        done = False
        doneEpisode = False
        t = 0
        state = env.reset()
        action = agent.act(state)
        actionA = [action]
        stateA = [state]
        rewardA = [0]
        while not done:
            state_next, reward, doneEpisode, info = env.step(action) 
            stateA.append(state_next)
            rewardA.append(reward)
            action_next = agent.act(state_next)
            actionA.append(action_next)
            agent.learn(state, action, reward, state_next, action_next, doneEpisode)
            action = action_next
            state  = state_next
            t += 1
            if t > max_nStep:
                done = True
            if doneEpisode == True:
                done = True
        total_reward_episodes[iEpisode] = np.sum(rewardA)
    return (total_reward_episodes, stateA)
    print('Finished')

## 4.3 Run the agent <a id="sec4_3">

We will run this agent in Mountain Car task where the goal is to pass to the right of the position point 0.5. Please see Example 10.1 for details of the environment.  

We create the environment,  prepare some parameters and create the agent. The learning parameter $\alpha$ is chosen using Figure 10.2. 

In [ ]:
# 'MountainCar-v0'  in default terminates the episodes with 200 steps. Calling with ".env" gets rid of this limitation.
#   A successful RL approach will be able to learn with 200 steps.  We allow changing the number of steps only for experimentation 
#   and to be able to make direct comparisons with Example 10.1 where maximum number of steps is at least 1000 
#   (You can look at Figure 10.2 to see this). 
env = gym.make('MountainCar-v0').env
print('State space:', env.observation_space)
print('Action space:', env.action_space)

# state limits for Mountain-Car Environment
min_position = -1.2
max_position = 0.6  # Gym 'MountainCar-v0' uses 0.6 instead of 0.5 of the book
max_velocity = 0.07
min_velocity = -0.07
stateRange = [max_position - min_position, max_velocity - min_velocity]

# Sampling of the state-space for cost-to-go plots
n_sample_plot = 60
posA_plot = np.linspace(min_position, max_position, n_sample_plot)
velA_plot = np.linspace(min_velocity, max_velocity, n_sample_plot)

agent = SARSA(gamma=1, epsilon=0, alpha=0.5/8, stateRange=stateRange)

We now train. __Note that training may take some time, wait until you see 'Finished'.__

In [ ]:
total_reward_episodes, stateA = train(env, agent, nEpisode = 500)

We now define some plot functions. 

In [ ]:
# Plot the position and velocity for MountainCar
def plotTrajectory(stateA):  
    posA = [state[0] for state in stateA]
    velA = [state[1] for state in stateA]

    plt.plot(posA, 'o')
    plt.xlabel('Steps')
    plt.ylabel('Position')
    plt.ylim(-1.3, 0.6)
    plt.grid(True)
    plt.show()

    plt.plot(velA, 'o')
    plt.xlabel('Steps')
    plt.ylabel('Velocity')
    plt.ylim(-0.075, 0.075)
    plt.grid(True)
    plt.show()

# Plot minus of total reward which corresponds to steps per episode for Mountain Car Environment
def plotStepsPerEpisode(total_reward_episodes): 
    plt.plot(-total_reward_episodes)
    plt.xlabel('Episode')
    plt.ylabel('Steps per episode')
    plt.ylim(0, 1000)
    plt.grid(True)
    plt.show()

We visually inspect the last trajectory. Has the car reached the goal? 



In [ ]:
plotTrajectory(stateA)

We plot the cost-to-go function learned in this run. Has the agent correctly identified the general behaviour of the expected number of steps before reaching the goal in different parts of state-space?

In [ ]:
agent.plot_costToGo(posA_plot, velA_plot, 'Position', 'Velocity')

We now plot the reward  during training. Note that the reward corresponds to total number of steps per episode. In terms of general trends, does the number steps per episode decrease during the training? 

In [ ]:
plotStepsPerEpisode(total_reward_episodes)

## 4.4 Discussions <a id="sec4_4">



We now compare our results with the figures in the book. Similar to the before, note the following: in this notebook we do not perform averaging over runs hence we typically have more randomness in our results compared to the book. We should keep this point in mind while interpreting our results. You are encouraged  to perform averaging over the runs after your initial experiments.

__Task-CD1:__  Similar to Figure 10.1, plot the cost-to-go function at different steps in the first episode and at the end of different episodes.  Verify that you can see the evolution of this function from a flat function of all zeros to functions similar to the ones Figure 10.1. You can create these plots using `agent.plot_costToGo(posA_plot, velA_plot, 'Position', 'Velocity')` at any point after creating the agent.

__Task-CD2:__  Consider Figure 10.2 which shows how the episode length varies with different learning rates as the agent trains. Try different $\alpha$ values for the agent. Do you observe the same type of behaviour? Explain the reason behind the general trend as $\alpha$ increases/decreases. 
